In [21]:
from pathlib import Path
from typing import List, Optional, Union, Literal, Tuple
from PIL import Image
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import InputFormat, DocumentStream
from docling.datamodel.pipeline_options import (
    AcceleratorDevice,
    AcceleratorOptions, 
    PdfPipelineOptions,
    smolvlm_picture_description
)
from docling_core.types.doc import PictureItem
import io
import base64
import json

In [2]:
class DocumentProcessor:
    def __init__(
        self,
        device: Optional[str] = None,
        num_threads: int = 8
    ):
        """
        Initialize the processor

        Args:
            device: Device for processing ('cuda', 'mps', 'cpu', or 'auto'). 
                If None, will use 'auto'.
            num_threads: Number of threads to use for processing
            picture_description: Type of picture description to use:
                - 'none': No picture description
                - 'smolVLM': Lightweight vision-language model
                - 'granite': Advanced vision-language model
            images_scale: Scale factor for images (default: 300/72.0)
        """
        device_map = {
            "cuda": AcceleratorDevice.CUDA,
            "mps": AcceleratorDevice.MPS, 
            "cpu": AcceleratorDevice.CPU,
            "auto": AcceleratorDevice.AUTO,
        }
        
        self.device = device or "auto"
        if self.device not in device_map:
            msg = f"Invalid device '{device}'. Must be one of: {list(device_map.keys())}"
            raise ValueError(msg)

        self.pipeline_options = PdfPipelineOptions(artifacts_path="/Users/georgestrong/multimodal-hybrid-parsing/models/v0.1.0/docling-models")
        self.pipeline_options.images_scale = 300/72.0
        self.pipeline_options.generate_page_images = True
        self.pipeline_options.generate_picture_images = True
        self.pipeline_options.accelerator_options = AcceleratorOptions(
            num_threads=num_threads,
            device=device_map[self.device]
        )
        self.pipeline_options.do_picture_description = True
        prompt = "Describe what you can see in this image. Focus only on what is visually present. Be concise and accurate in three sentences maximum."
        self.pipeline_options.picture_description_options = smolvlm_picture_description
        self.pipeline_options.picture_description_options.prompt = prompt
        self.pipeline_options.picture_description_options.generation_config = {
            "max_new_tokens": 500,
            "do_sample": False,
        }

        self.converter = DocumentConverter(
            format_options={
                InputFormat.PDF: PdfFormatOption(
                    pipeline_options=self.pipeline_options
                )
            }
        )

    def __call__(
        self, 
        file_path: Union[str, Path]
    ) -> Tuple[List[str], List[Image.Image], List[Image.Image]]:
        """
        Process a document and return markdown pages and images

        Args:
            file_path: Path to the PDF file

        Returns:
            Tuple containing:
            - List of markdown strings (one per page)
            - List of extracted images (figures/graphs)
            - List of page images
        """
        file_path = Path(file_path) if isinstance(file_path, str) else file_path

        result = self.converter.convert(file_path)

        page_images = []
        extracted_images = []
        markdown_pages = []
        picture_descriptions = {}

        # Get page images from the conversion result
        for page_no, page in result.document.pages.items():
            if hasattr(page, 'image') and page.image is not None:
                page_images.append(page.image.pil_image)

        # Collect images and descriptions
        for element, _level in result.document.iterate_items():
            if isinstance(element, PictureItem):
                page_no = element.prov[0].page_no
                
                if hasattr(element, 'image') and element.image is not None:
                    extracted_images.append(element.image.pil_image)
                
                if page_no not in picture_descriptions:
                    picture_descriptions[page_no] = []
                
                if element.annotations:
                    ann = element.annotations[0]
                    desc = f"**AI-Generated Image Description:** {ann.text}\n<!-- end image description -->"
                    picture_descriptions[page_no].append(desc)

        # Process markdown pages
        for i in range(result.document.num_pages()):
            page_no = i + 1
            page_md = result.document.export_to_markdown(page_no=page_no)
            
            if page_no in picture_descriptions:
                parts = page_md.split("<!-- image -->")
                new_page_md = parts[0]
                
                for idx, part in enumerate(parts[1:]):
                    if idx < len(picture_descriptions[page_no]):
                        description = picture_descriptions[page_no][idx]
                        new_page_md += f"<!-- image -->\n{description}\n{part}"
                    else:
                        new_page_md += f"<!-- image -->{part}"
                
                page_md = new_page_md
            
            markdown_pages.append(page_md)

        return markdown_pages, extracted_images, page_images

In [3]:
class DocumentProcessor:
    def __init__(
        self,
    ):
        self.pipeline_options = PdfPipelineOptions()
        self.pipeline_options.images_scale = 300/72.0
        self.pipeline_options.generate_page_images = True
        self.pipeline_options.generate_picture_images = True
        self.pipeline_options.accelerator_options = AcceleratorOptions(
            num_threads=8,
            device=AcceleratorDevice.CUDA
        )
        self.pipeline_options.do_picture_description = False
        prompt = "Describe what you can see in this image. Focus only on what is visually present. Be concise and accurate in three sentences maximum."
        self.pipeline_options.picture_description_options = smolvlm_picture_description
        self.pipeline_options.picture_description_options.prompt = prompt
        self.pipeline_options.picture_description_options.generation_config = {
            "max_new_tokens": 500,
            "do_sample": False,
        }

        self.converter = DocumentConverter(
            format_options={
                InputFormat.PDF: PdfFormatOption(
                    pipeline_options=self.pipeline_options
                )
            }
        )

    def __call__(
        self, 
        base64_content: str
    ) -> Tuple[List[str], List[str], List[str]]:
        """
        Process a document and return markdown pages and base64 encoded images

        Args:
            base64_content: Base64 encoded PDF file content

        Returns:
            Tuple containing:
            - List of markdown strings (one per page)
            - List of base64 encoded extracted images (figures/graphs)
            - List of base64 encoded page images
        """
        # Decode base64 content to bytes
        pdf_content = base64.b64decode(base64_content)
        
        # Create BytesIO object and DocumentStream
        pdf_stream = io.BytesIO(pdf_content)
        source = DocumentStream(name="doc.pdf", stream=pdf_stream)

        # Convert using DocumentStream
        result = self.converter.convert(source)

        page_images = []
        extracted_images = []
        markdown_pages = []
        picture_descriptions = {}

        # Get page images from the conversion result
        for page_no, page in result.document.pages.items():
            if hasattr(page, 'image') and page.image is not None:
                page_images.append(str(page.image.uri))

        # Collect images and descriptions
        for element, _level in result.document.iterate_items():
            if isinstance(element, PictureItem):
                page_no = element.prov[0].page_no
                
                if hasattr(element, 'image') and element.image is not None:
                    extracted_images.append(str(element.image.uri))
                
                if page_no not in picture_descriptions:
                    picture_descriptions[page_no] = []
                
                if element.annotations:
                    ann = element.annotations[0]
                    desc = f"**AI-Generated Image Description:** {ann.text}\n<!-- end image description -->"
                    picture_descriptions[page_no].append(desc)

        # Process markdown pages
        for i in range(result.document.num_pages()):
            page_no = i + 1
            page_md = result.document.export_to_markdown(page_no=page_no)
            
            if page_no in picture_descriptions:
                parts = page_md.split("<!-- image -->")
                new_page_md = parts[0]
                
                for idx, part in enumerate(parts[1:]):
                    if idx < len(picture_descriptions[page_no]):
                        description = picture_descriptions[page_no][idx]
                        new_page_md += f"<!-- image -->\n{description}\n{part}"
                    else:
                        new_page_md += f"<!-- image -->{part}"
                
                page_md = new_page_md
            
            markdown_pages.append(page_md)

        return markdown_pages, extracted_images, page_images

In [4]:
doc_processor = DocumentProcessor()

In [23]:
import base64

with open("samples/[table] Black White Minimalist Simple Creative Freelancer Invoice (1).pdf", 'rb') as pdf_file:
    encoded_string = base64.b64encode(pdf_file.read())
    base64_string = encoded_string.decode('utf-8')

In [20]:
output_file = "../models/v0.1.0/document_base64.txt"
with open(output_file, "w") as f:
    f.write('"'+base64_string+'"')

In [ ]:
output_file = "../models/v0.1.0/sample_payload.json"

# Create a JSON payload
payload = {"pdf_content": base64_string}

# Write the JSON payload to a file
with open(output_file, "w") as f:
    json.dump(payload, f)

In [ ]:
markdown_pages, extracted_images, page_images = doc_processor(base64_string)

In [7]:
doc_processor = DocumentProcessor(
    device="cpu",
    num_threads=8,
)

In [8]:
markdown_pages, extracted_images, page_images = doc_processor("samples/[research-paper] 1312.6114v11.pdf")

In [9]:
markdown_pages[0]

'## Auto-Encoding Variational Bayes\n\nDiederik P. Kingma Machine Learning Group Universiteit van Amsterdam dpkingma@gmail.com\n\n## Max Welling\n\nMachine Learning Group Universiteit van Amsterdam welling.max@gmail.com\n\n## Abstract\n\nHow can we perform efficient inference and learning in directed probabilistic models, in the presence of continuous latent variables with intractable posterior distributions, and large datasets? We introduce a stochastic variational inference and learning algorithm that scales to large datasets and, under some mild differentiability conditions, even works in the intractable case. Our contributions are two-fold. First, we show that a reparameterization of the variational lower bound yields a lower bound estimator that can be straightforwardly optimized using standard stochastic gradient methods. Second, we show that for i.i.d. datasets with continuous latent variables per datapoint, posterior inference can be made especially efficient by fitting an appr